### 0) 초기 패키지 설치 및 데이터 로드

In [ ]:
!pip install fugashi
!pip install sentencepiece
!pip install unidic-lite
!pip install transformers
!pip install torch
!pip install torchvision
!pip install spacy
# !python -m spacy download ja_core_news_sm
!pip install mecab-python3
!pip install ja_stopword_remover #stopwords
!pip install neologdn
!pip install demoji
!pip install mojimoji
# !pip install -U ginza ja_ginza

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties #시각화 시 일본어 폰트 지원
fp = FontProperties(fname=r'/content/drive/MyDrive/BA_capstone/ヒラギノ丸ゴ ProN W4.ttc', size=12)

df = pd.read_csv('/content/drive/MyDrive/BA_capstone/qna_df_0928.csv',encoding='utf-8')
df

### 1) 텍스트 데이터 전처리

In [ ]:
import re
import mojimoji as mo
import MeCab
from ja_stopword_remover.remover import StopwordRemover
import neologdn
import demoji

In [ ]:
#@title 단어 치환  함수 정의 1
#제거하지 않을 영어 단어 지정
exclude_words = ['ems', 'led', 'uv','aha','bha','youtube','youtuber','tiktok',
                 'instagram','spf','cnp','kanebo','3ce','sum37','lush']

def special_text_replace(X):
  replacements = {
        ##뷰티관련##
        '@コスメ':'アットコスメ','@cosme':'アットコスメ',
        'おすすめ': 'オススメ',
        'ドラコス': 'プチプラ',
        'ベスコス': 'ベストコスメアワード',
        'プチプラコスメ': 'プチプラ',
        'デパートコスメ': 'デパコス',
        '口コミ': 'クチコミ',
        'hifu':'ハイフ',            ##hifu lazer
        'しみ':'シミ','染み':'シミ','染(?!み)':'シミ',#기미
        'しわ':'シワ','皺':'シワ', #주름
        'ユーチューブ':'youtube','ユーチューバー':'youtuber','youtuber':'youtube',
        'ティックトック':'tiktok','tik tok':'tiktok',
        'insta(?!gram)':'instagram','インスタグラム':'instagram','インスタ(?!グラム)':'instagram',
        'refa':'リファ',
        'daily':'日常','store':'ストア',
        'オイリー肌':'脂性肌','油脂':'オイル','油(?!脂)':'オイル','(?<!インナー)ドライ肌':'乾燥肌',
        'リピ(?!ート)':'リピート',
        'バームクレンジング':'クレンジングバーム',
        'クレンジングフォーム':'洗顔フォーム',
        'クレンジングオイル':'オイルクレンジング',
        'クレンジングミルク':'ミルククレンジング','ミルク洗顔':'ミルククレンジング',
        'ジェル洗顔':'ジェルクレンジング',
        'アイリムーバー':'アイメイクアップリムーバー',
        'ファンデ(?!ーション)':'ファンデーション',
        '用品':'商品','製品':'商品','アイテム':'商品',
        '(?<!アット)コスメ':'化粧品','実感':'効果',
        '(?<!ハンド)ソープ':'石鹸','花粉(?!症)':'花粉症',
        'ダブルクレンジング':'ダブル洗顔',
        '(?<!キャン)メイク':'メイクアップ','基礎化粧品':'スキンケア商品',
        'テカる':'テカリ','テカテカ':'テカリ',

        ##기타 22
        '次の日':'翌日','役割':'機能',
        '先生':'医師','医者':'医師','ボディー':'ボディ','身体':'ボディ','全身':'ボディ',
        '(?<!副)作用':'効果','女子':'女性','手順':'順番',
        '食事':'食生活','食べ物':'食生活','食品':'食生活',
        '試供品':'サンプル','トライアル':'サンプル','使い心地':'使用感',
        'やり方':'仕方','種類':'タイプ','シートマスク':'シートパック',
        'よろしくお願いします':'',




        ##스킨케어 관련##
        '潤い': 'うるおい',#촉촉함
        'にきび': 'ニキビ', #여드름

        ###쇼핑몰####
        'qoo10': 'キュウテン',
        'amazon': 'アマゾン',
        'rakuten': '楽天',
        '無印(?!良品)': '無印良品',
        'ds':'ドラックストア',
        'daiso':'ダイソー',


        ###데파코스 브랜드#####
        'addiction': 'アディクション','アディクションさん': 'アディクション',
        'annasui': 'アナスイ','アナスイさん': 'アナスイ',
        'albion': 'アルビオン','アルビオンさん': 'アルビオン',
        'rmk': 'アールエムケー','アールエムケーさん': 'アールエムケー',
        'ipsa':'イプサ','イプサさん': 'イプサ',
        'ysl':'イヴサンローラン','(?<!イヴ)サンローラン':'イヴサンローラン','yves saint laurent':'イヴサンローラン', 'イヴサンローランさん':'イヴサンローラン',
        'skii':'エスケーツー','sk2':'エスケーツー','skⅱ':'エスケーツー','sk-ⅱ':'エスケーツー','sk-ii':'エスケーツー','sk-2':'エスケーツー','エスケーツーさん':'エスケーツー',
        'esteelauder':'エスティローダー','estee lauder':'エスティローダー','エスティローダーさん':'エスティローダー','エスティーローダー':'エスティローダー',
        'est(?!eelauder)':'エスト','エストさん':'エスト',
        'etvos':'エトヴォス','エトヴォスさん':'エトヴォス',
        'orlane':'オルラーヌ','オルラーヌさん':'オルラーヌ',
        'chicca':'キッカ','キッカさん':'キッカ',
        'clarins':'クラランス','クラランスさん':'クラランス',
        'clinique':'クリニーク','クリニークさん':'クリニーク',
        'cledepeaubeaute':'クレドポーボーテ','クレド(?!ポーボーテ)':'クレドポーボーテ', 'クレドポーボーテさん':'クレドポーボーテ',
        'gucci':'グッチ','グッチさん':'グッチ',
        'kesalanpatharan':'ケサランパサラン', 'ケサランパサランさん':'ケサランパサラン',
        'guerlain':'ゲラン', 'ゲランさん':'ゲラン',
        'cosmedecorte':'コスメデコルテ','(?<!cosme)decorte':'コスメデコルテ','コスデコ':'コスメデコルテ','(?<!コスメ)デコルテ':'コスメデコルテ','コスメデコルテさん':'コスメデコルテ',
        'chanel': 'シャネル', 'シャネルさん':'シャネル',
        'shu uemura':'シュウウエムラ','shuuemura':'シュウウエムラ','シュウ(?!ウエムラ)':'シュウウエムラ','シュウウエムラさん':'シュウウエムラ',
        'givenchy':'ジバンシィ','ジバンシィさん':'ジバンシィ',
        'giorgio armani':'ジョルジオアルマーニ','(?<!giorgio)armani':'ジョルジオアルマーニ','(?<!ジョルジオ)アルマーニ':'ジョルジオアルマーニ','ジョルジオアルマーニさん':'ジョルジオアルマーニ',
        'jillstuart':'ジルスチュアート','ジル(?!スチュアート)':'ジルスチュアート', 'ジルスチュアートさん':'ジルスチュアート',
        'suqqu':'スック', 'スックさん':'スック',
        'nars': 'ナーズ','ナーズさん': 'ナーズ',
        'shiseido': '資生堂','資生堂さん': '資生堂',
        'three':'スリー','スリーさん':'スリー',
        'dazzshop':'ダズショップ','ダズショップさん':'ダズショップ',
        'dior':'ディオール','ディオールさん':'ディオール','カプチュール(?!トータルルセラム)':'カプチュールトータル',
        'd if story':'ディフストーリー','d.if story':'ディフストーリー','ディフストーリーさん':'ディフストーリー',
        'too faced':'トゥーフェイスド','toofaced':'トゥーフェイスド','トゥーフェイスドさん':'トゥーフェイスド',
        'tomford': 'トムフォード','トム(?!フォード)': 'トムフォード','トムフォードさん': 'トムフォード','(?<!ハ)トム(?!ギ)': 'トムフォード','(?<!ハ)トム': 'トムフォード','トム(?!ギ)': 'トムフォード',
        'delamer': 'ドゥラメール','ドゥラメールさん': 'ドゥラメール',
        'noevir': 'ノエビアスペチアーレ','ノエビア(?!スペチアーレ)': 'ノエビアスペチアーレ','ノエビアスペチアーレさん': 'ノエビアスペチアーレ',
        'burberry': 'バーバリー','バーバリーさん': 'バーバリー',
        'bobbi brown': 'ボビィブラウン','ボビィ(?!ブラウン)': 'ボビィブラウン','ボビィブラウンさん': 'ボビィブラウン',
        'pola': 'ポーラ','ポーラさん': 'ポーラ',
        'paul&joe': 'ポールアンドジョー','p&j': 'ポールアンドジョー','ポール&ジョー': 'ポールアンドジョー', 'ポルジョ': 'ポールアンドジョー', 'ポールアンドジョーさん': 'ポールアンドジョー',
        'mac': 'マック','マックさん': 'マック',
        'lancome': 'ランコム','ランコムさん': 'ランコム',
        'lunasol': 'ルナソル','ルナソルさん': 'ルナソル',
        'loccitane': 'ロクシタン','l’occitane': 'ロクシタン','ロクシタンさん': 'ロクシタン',
        'laura mercier': 'ローラメルシエ','ローラ(?!メルシエ)': 'ローラメルシエ','ローラメルシエさん': 'ローラメルシエ',
        'les merveilleuses laduree': 'レメルヴェイユーズラデュレ','(?<!レメルヴェイユーズ)ラデュレ': 'レメルヴェイユーズラデュレ','レメルヴェイユーズラデュレさん': 'レメルヴェイユーズラデュレ',
        'makeupforever': 'メイクアップフォーエバー','mufe': 'メイクアップフォーエバー','メイクアップフォーエバーさん': 'メイクアップフォーエバー',
        'elegance': 'エレガンス','エレガンスさん': 'エレガンス',
        'celvoke': 'セルヴォーク','セルヴォークさん': 'セルヴォーク',
        'kiehls': 'キールズ','キールズさん': 'キールズ',
        'dolce&gabana': 'ドルチェアンドガッバーナ','ドルチェ&ガッバーナ': 'ドルチェアンドガッバーナ','ドルガバ': 'ドルチェアンドガッバーナ','ドルチェアンドガッバーナさん': 'ドルチェアンドガッバーナ',
        'shiro':'シロ','シロさん':'シロ',

        'kose':'コーセー','dプログラム':'ディープログラム',


        ####기타 화장품 브랜드####
        'skin&lab(スキンアンドラブ)': 'スキンアンドラブ','skin&lab(?!(スキンアンドラブ))': 'スキンアンドラブ','スキンアンドラブさん': 'スキンアンドラブ',
        'vt': 'ブイティー','ブイティーさん': 'ブイティー',
        'duo': 'デュオ','デュオさん': 'デュオ',
        'aqualabel': 'アクアレーベル','アクアレーベルさん': 'アクアレーベル',
        'haku': 'ハク','ハクさん': 'ハク',
        'ba': 'ビーエー','b.a': 'ビーエー','ビーエーさん': 'ビーエー',
        'ahres': 'アーレス','アーレスさん': 'アーレス',
        'elixir': 'エリクシール','エリクシールさん': 'エリクシール',
        'predia': 'プレディア','プレディアさん': 'プレディア',
        'alblanc': 'アルブラン','アルブランさん': 'アルブラン',
        'branchic': 'ブランシック','ブランシックさん': 'ブランシック',
        'dejavu': 'デジャヴュ','デジャヴュさん': 'デジャヴュ',
        'obagi': 'オバジ','オバジさん': 'オバジ',
        'canmake': 'キャンメイク','キャンメイクさん': 'キャンメイク',
        'javin de seoul': 'ジャビンドゥソウル','ジャビンドゥソウルさん': 'ジャビンドゥソウル',
        'visee': 'ヴィセ','ヴィセさん': 'ヴィセ',
        'dew': 'デュウ','デュウさん': 'デュウ',
        'dr.ci:labo': 'ドクターシーラボ','dr.ci labo': 'ドクターシーラボ','ドクターシーラボさん': 'ドクターシーラボ',
        'kate':'ケイト','ケイトさん': 'ケイト',
        'excel': 'エクセル','エクセルさん': 'エクセル',
        'astalift': 'アスタリフト','アスタリフトさん': 'アスタリフト',
        'sabon': 'サボン','サボンさん': 'サボン',
        'fancl': 'ファンケル','ファンケルさん': 'ファンケル',
        'ettusais': 'エテュセ','エテュセさん': 'エテュセ',
        'n organic': 'エヌオーガニック','エヌオーガニックさん': 'エヌオーガニック',
        'electron': 'エレクトロン','エレクトロンさん': 'エレクトロン',
        'mentholatum': 'メンソレータム','メンソレータムさん': 'メンソレータム',
        'rosyrosa': 'ロージーローザ','ロージーローザさん': 'ロージーローザ',
        'FAS': 'ファス','ファスさん': 'ファス',
        'APLIN': 'アプリン','アプリンさん': 'アプリン',
        'cezanne': 'セザンヌ','セザンヌさん': 'セザンヌ',
        '肌研': 'ハダラボ','肌ラボ': 'ハダラボ','ハダラボさん': 'ハダラボ',
        'chacott cosmetics': 'チャコットコスメティクス','chacottcosmetics': 'チャコットコスメティクス','チャコット': 'チャコットコスメティクス','チャコットコスメティクスさん': 'チャコットコスメティクス',
        'apagard': 'アパガード','アパガードさん': 'アパガード',
        'attenir': 'アテニア','アテニアさん': 'アテニア',
        'shiroru': 'シロル','シロルさん': 'シロル',
        'primavista': 'プリマヴィスタ','プリマヴィスタさん': 'プリマヴィスタ',
        'sofina': 'ソフィーナ','ソフィーナさん': 'ソフィーナ',
        'the collagen': 'ザコラーゲン','ザコラーゲンさん': 'ザコラーゲン',
        'sebamed': 'セバメド','セバメドさん': 'セバメド',
        'atrix': 'アトリックス','アトリックスさん': 'アトリックス',
        'herris': 'ハーリス','ハーリスさん': 'ハーリス',
        'saborino': 'サボリーノ','サボリーノさん': 'サボリーノ',
        'amplitude': 'アンプリチュード','アンプリチュードさん': 'アンプリチュード',
        'house of rose':'ハウスオブローゼ','ハウスオブローゼさん':'ハウスオブローゼ',
        'la roche-posay':'ラロッシュポゼ','larocheposay':'ラロッシュポゼ','la roche posay':'ラロッシュポゼ','ラロッシュ(?!ポゼ)':'ラロッシュポゼ','ラロッシュポゼさん':'ラロッシュポゼ',
        'lululun':'ルルルン','ルルルンさん':'ルルルン',
        'メラノcc':'メラノシーシー','メラノシーシーさん':'メラノシーシー',
        'suisavon':'首里石鹸','首里石鹸さん':'首里石鹸',
        'ink':'インク','インクさん':'インク',
        'ignis':'イグニス','イグニスさん':'イグニス',
        'curel':'キュレル','キュレルさん':'キュレル',
        'apex':'アペックス','アペックスさん':'アペックス',
        'ラッシュ':'lush','カネボウ':'kanebo',



        ##한국브랜드##
        'the history of whoo': 'ドフー','thehistoryofwhoo': 'ドフー','thehistoryof后': 'ドフー','the history of 后': 'ドフー','ザヒストリーオブフー': 'ドフー','ザフー': 'ドフー','ドフーさん': 'ドフー',
        'innisfree':'イニスフリー','イニスフリーさん':'イニスフリー',
        'hera': 'ヘラ','ヘラさん': 'ヘラ',
        'etude house':'エチュードハウス','etudehouse':'エチュードハウス','エチュードハウスさん':'エチュードハウス',
        'the saem':'ザセム','ザセムさん':'ザセム',
        'iope':'アイオペ','アイオペさん':'アイオペ',
        'tonymoly':'トニーモリー','トニーモリーさん':'トニーモリー',
        'missha':'ミシャ','ミシャさん':'ミシャ',
        'laneige':'ラネージュ','ラネージュさん':'ラネージュ',
        'mamonde':'マモンド','マモンドさん':'マモンド',
        'apieu':'アピュー','アピューさん':'アピュー',
        'hanyul':'ハンユル','ハンユルさん':'ハンユル',
        'nature republic':'ネイチャーリパブリック','naturerepublic':'ネイチャーリパブリック','ネイチャーリパブリックさん':'ネイチャーリパブリック',
        'the face shop':'ザフェイスショップ','thefaceshop':'ザフェイスショップ','ザフェイスショップさん':'ザフェイスショップ',
        'primera':'プリメラ','プリメラさん':'プリメラ',
        'skinfood':'スキンフード','スキンフードさん':'スキンフード',
        'itsskin':'イッツスキン','イッツスキンさん':'イッツスキン',
        'sulwhasoo':'ソルファス','雪花秀':'ソルファス','ソルファスさん':'ソルファス',
        'wonjungyo': 'ウォンジョンヨ','ウォンジョンヨさん': 'ウォンジョンヨ',
        'abib': 'アビブ','アビブさん': 'アビブ',
        'cosrx': 'コスアールエックス','コスアールエックスさん': 'コスアールエックス',
        'tirtir': 'ティルティル','ティルティルさん': 'ティルティル',
        'anua': 'アヌア','アヌアさん': 'アヌア',
        'aritaum': 'アリタウム','アリタウムさん': 'アリタウム',
        'beyond': 'ビヨンド','ビヨンドさん': 'ビヨンド',
        'sooryehan': 'スリョハン','スリョハンさん': 'スリョハン',
        'holikaholika': 'ホリカホリカ','holika holika': 'ホリカホリカ','ホリカホリカさん': 'ホリカホリカ',
        'banilaco': 'バニラコ','banila co': 'バニラコ','バニラコさん': 'バニラコ',
        'goodal': 'グーダル','グーダルさん': 'グーダル',
        'clio': 'クリオ','クリオさん': 'クリオ',
        'peripera': 'ペリペラ','ペリペラさん': 'ペリペラ',
        'mediheal': 'メディヒール','メディヒールさん': 'メディヒール',
        'espoir': 'エスポア','エスポアさん': 'エスポア',
        'too cool for school': 'トゥークールフォースクール','toocoolforschool': 'トゥークールフォースクール',
        'dr.jart': 'ドクタージャルト','drjart': 'ドクタージャルト','ドクタージャルトさん': 'ドクタージャルト',
        'april skin': 'エイプリルスキン','aprilskin': 'エイプリルスキン','エイプリルスキンさん': 'エイプリルスキン',
        'labiotte': 'ラビオッテ','ラビオッテさん': 'ラビオッテ', ##토니모리의 고급브랜드
        'belief': 'ビリーフ','ビリーフさん': 'ビリーフ',
        'cnp laboratory': 'cnp','cnplaboratory': 'cnp','チャアンドパク': 'cnp','cnpさん': 'cnp',
        'bring green': 'ブリングリーン','bringgreen': 'ブリングリーン','ブリングリーンさん': 'ブリングリーン',
        'jung saem mool': 'ジョンセンムル','jungsaemmool': 'ジョンセンムル','ジョンセンムルさん': 'ジョンセンムル',
        'numbuzin':'ナンバーズイン','ナンバーズインさん':'ナンバーズイン',
        'manyo':'魔女工場','マニョ':'魔女工場','魔女工場さん':'魔女工場',
        'torriden': 'トリデン','トリデンさん': 'トリデン',
        'suum37':'sum37','スム37':'sum37','sum37さん':'sum37',
    }

  for key, value in replacements.items():
    X = re.sub(key, value, X,flags=re.IGNORECASE)  # 대소문자 구분 없이 변환

  return X


In [ ]:
#@title 단어 치환 함수 정의 2

def special_text_cleaning(X):

    # 문자열이 아닌 경우 처리를 건너뛰기
    if not isinstance(X, str):
        return X

    # 반각문자를 전각문자로 변환(문자 종류 통일)
    def change_kana(text):
        x = mo.zen_to_han(text, kana=False) # 영어 전각을 반각으로
        # x = mo.han_to_zen(x, ascii=False)  # 카나 반각을 전각으로
        x = x.lower()  # 영어 대문자를 소문자로
        return x

    ##상단의 함수 적용
    tmp = change_kana(X)  # 문자종류 통일 / 영어 소문자화

    # 다양한 특수문자, 이모티콘, URL 제거
    patterns = [
        r'[\t\s!"#$%&\'\\\\()*+,-./:;；：<=>?@[\\]^_`{|}~○｢｣「」〔〕“”〈〉『』&#8203;``oaicite:{"number":1,"invalid_reason":"Malformed citation &#8203;``oaicite:{"number":1,"invalid_reason":"Malformed citation 【】"}``&#8203;"}``&#8203;＆＊（）＄＃＠？！｀＋￥¥％♪…◇→←↓↑｡･ω･｡ﾟ´∀｀ΣДｘ⑥◎©︎♡★☆▽※ゞノ〆εσ＞＜┌┘]',
        r'[!-/:-@[-`{-~]',  # ASCII 기호
        r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+',  # URL
        r'[■-♯]',  # 일부 특수 문자
        r'①|②|´|・|ω|・|｀|ヽ'  # 기타 문자
    ]
    for pattern in patterns:
        tmp = re.sub(pattern, '', tmp)

    tmp = special_text_replace(tmp)  # 특정 단어 치환

    return tmp

In [ ]:
#@title 단어 정규화함수

def only_letters(text):
    # スム37을 일시적으로 치환
    text = text.replace("sum37", "TMPPLACEHOLDER")
    text = text.replace("tゾーン", "TMPPLACEHOLDER_second")
    text = mo.han_to_zen(text, ascii=False)  # 카나 반각을 전각으로
    text = re.sub(r'(\d{1,3}(,\d{3})*|\d+)', '0', text)  # 쉼표가 있는 숫자도 대체
    text = text.replace('\n', '').replace('\r', '')  # 줄바꿈 제거
    text = demoji.replace(string=text, repl='')  # 이모티콘 제거

    # 일시적으로 치환한 스무37을 원래대로 복구
    text = text.replace("TMPPLACEHOLDER", "sum37")
    text = text.replace("TMPPLACEHOLDER_second", "tゾーン")

    return text

def remove_english_except_exclusions(text, exclude_words):
    # 알파벳 문자열을 찾는 정규 표현식 패턴
    english_pattern = re.compile(r'[A-Za-z]+')

    # 텍스트 내의 모든 영어 단어를 찾습니다.
    english_words = re.findall(english_pattern, text)

    # 제외 단어 목록에 없는 단어만 제거
    for word in english_words:
        if word.lower() not in [e.lower() for e in exclude_words]:
            text = text.replace(word, '')

    return text



def text_normalization(X):
    # 문자열이 아닌 경우 처리를 건너뛰기
    if not isinstance(X, str):
        return X

    tmp = neologdn.normalize(X)  # 정규화
    tmp = only_letters(tmp)
    tmp = remove_english_except_exclusions(tmp,exclude_words) #영어제거

    # 다양한 특수문자, 이모티콘, URL 제거
    patterns = [
        r'[\t\s!"#$%&\'\\\\()*+,-./:;；：<=>?@[\\]^_`{|}~○｢｣「」〔〕“”〈〉『』&#8203;``oaicite:{"number":1,"invalid_reason":"Malformed citation &#8203;``oaicite:{"number":1,"invalid_reason":"Malformed citation 【】"}``&#8203;"}``&#8203;＆＊（）＄＃＠？！｀＋￥¥％♪…◇→←↓↑｡･ω･｡ﾟ´∀｀ΣДｘ⑥◎©︎♡★☆▽※ゞノ〆εσ＞＜┌┘]',
        r'[!-/:-@[-`{-~]',  # ASCII 기호
        r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+',  # URL
        r'[■-♯]',  # 일부 특수 문자
        r'①|②|´|・|ω|・|｀|ヽ|【|】'  # 기타 문자
    ]
    for pattern in patterns:
        tmp = re.sub(pattern, '', tmp)

    return tmp


In [ ]:
#전처리 함수 적용

def preprocess_text(text):
  if not isinstance(text, str):
        return text

  text = special_text_cleaning(text)
  text = text_normalization(text)
  return text

#0번째 전처리 함수 적용
df2 =  df.copy()
# df2['내용'] = df2['내용'].apply(special_text_cleaning)

#전처리 함수 적용
df2['내용'] = df2['내용'].apply(preprocess_text)

# df2.to_csv('./normalized_content_v6.csv', index=False,encoding='utf-8-sig') #저장

### 2) MeCab 사용자 사전 적용

In [ ]:
!pip install mecab-python3 unidic-lite
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
import subprocess

# MeCab의 설치 경로를 찾습니다.
mecab_path = subprocess.getoutput("which mecab")
mecab_dic_path = subprocess.getoutput("mecab-config --dicdir")

print("MeCab Path:", mecab_path)
print("MeCab Dic Path:", mecab_dic_path)

MeCab Path: /usr/bin/mecab
MeCab Dic Path: /usr/lib/x86_64-linux-gnu/mecab/dic


In [ ]:
import subprocess

def find_file_path(filename):
    try:
        # 파일을 찾기 위한 명령어를 구성합니다. 최상위 디렉토리부터 검색을 시작합니다.
        command = f"find / -type f -name {filename} 2>/dev/null"

        # 명령을 실행합니다.
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)

        # 명령의 출력 결과를 받습니다.
        output, error = process.communicate()

        # 'output' 변수에 결과가 담겨 있습니다.
        if output:
            # 파일 경로를 출력합니다. 여러 경로가 있을 수 있으니, 모든 결과를 확인하세요.
            print(f"'{filename}' 파일을 찾았습니다. 경로는 다음과 같습니다:")
            print(output)
        else:
            print(f"'{filename}' 파일을 찾지 못했습니다.")

    except Exception as e:
        print(e)

# 사용 예시
find_file_path("mecab-dict-index")

'mecab-dict-index' 파일을 찾았습니다. 경로는 다음과 같습니다:
/usr/lib/mecab/mecab-dict-index



In [ ]:
!mkdir ../var/lib/mecab/dic/userdic

In [ ]:
!/usr/lib/mecab/mecab-dict-index \
-d /usr/share/mecab/dic/ipadic \
-u /content/userdic.dic \
-f utf-8-sig \
-t utf-8-sig "/content/user_dic_mecab2.csv"

reading /content/user_dic_mecab2.csv ... 277
emitting double-array: 100% |###########################################| 

done!


In [ ]:
!echo "userdic = /content/userdic.dic" >> /etc/mecabrc

In [ ]:
#사용자 사전 작동확인
import MeCab
tagger = MeCab.Tagger('-O wakati')
print(tagger.parse('コスメデコルテが好きだよ。資生堂のオイルクレンジングも必要！'))
print(tagger.parse('私はインナードライ肌でとにかく敏感肌です。すぐにニキビや吹き出物ができ、毛穴の開きも目立ちま...	'))

### 3) 버토픽 실행

In [ ]:
!pip install bertopic

In [ ]:
#전처리 된 텍스트 로드
pre_df = pd.read_csv('/content/drive/MyDrive/BA_capstone/normalized_content_v6.csv',encoding='utf-8')
pre_df

In [ ]:
##컬럼 지정
pre_df2 = pre_df[pre_df['내용'].apply(lambda x: isinstance(x, str))]
text = pre_df2['내용']
text = text.reset_index(drop=True)

# '날짜' 열의 데이터를 pandas 날짜 형식으로 변환
pre_df2['날짜'] = pd.to_datetime(pre_df2['날짜'])
dates_list = pre_df2['날짜'].tolist()

<ipython-input-15-8f52b2515957>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_df2['날짜'] = pd.to_datetime(pre_df2['날짜'])


In [ ]:
#@title 토큰화 함수_명사
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import MeCab
import pandas as pd

def tokenize_jp_v2(text,unique_values):
    if not isinstance(text, str):
        return text

    with open('/content/drive/MyDrive/BA_capstone/stop_words.txt', 'r', encoding='utf-8') as f:
        stopwords = set(f.read().splitlines())
    with open('/content/drive/MyDrive/BA_capstone/my_stopwords.txt', 'r', encoding='utf-8') as f:
        new_stopwords = set(f.read().splitlines())


    # MeCab를 사용하여 텍스트를 토큰화하고 형태소 분석합니다.
    mecab = MeCab.Tagger("-Ochasen")
    node = mecab.parseToNode(text)
    words = []

    while node:
        features = node.feature.split(',')
        if features[0] == '名詞' and (features[1] == '一般' or features[1] == '固有名詞') or node.surface in unique_values:
            words.append(node.surface)
        node = node.next

    not_need_words = ['。','ください','です','コイン','ます','スタンプ','お過ごし','皆様','皆さま','皆さん',
                      'ます','くらい','ぐらい','さん','押し','ましょ','よく','今日','教えてください','お願いいたします']

    # 불용어를 제거합니다.
    words = [w for w in words if w not in stopwords and w not in new_stopwords and w not in not_need_words]
    words = [x for x in words if len(x) > 1 or x in ['朝', '夜']] # '肌'
    words = [x for x in words if len(x) < 10]

    return words

# tokenized_text = tokenize_jp_v2(your_text, unique_values)

In [ ]:
#@title 버토픽 실행 후 저장
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
import hdbscan
from umap import UMAP
from bertopic.vectorizers import ClassTfidfTransformer

# KeyBERT
keybert_model = KeyBERTInspired()

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.7)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model
}
#cluster_selection_method='eom', ,min_samples = 10
# hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10, metric='euclidean',prediction_data=True)


# umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine',random_state=42)

from functools import partial

# 기존의 tokenizer 함수를 partial을 사용하여 unique_values를 추가합니다.
tokenize_with_unique_values = partial(tokenize_jp_v2, unique_values=unique_values)

# 이렇게 수정된 tokenizer를 CountVectorizer에 넣습니다.
vectorizer = CountVectorizer(tokenizer=tokenize_with_unique_values)

# ctfidf_model = ClassTfidfTransformer()

from bertopic import BERTopic    ###nr_topics=20 ## min_topic_size=10 umap_model=umap_model,,hdbscan_model=hdbscan_model
topic_model = BERTopic(min_topic_size=50,nr_topics=50,language="japanese", representation_model=representation_model,
                       calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer)
topics, probs = topic_model.fit_transform(text)

topic_model.save("my_model", serialization="pickle") #버토픽모델저장

Batches:   0%|          | 0/891 [00:00<?, ?it/s]

2023-11-22 08:29:57,879 - BERTopic - Transformed documents to Embeddings
2023-11-22 08:30:12,865 - BERTopic - Reduced dimensionality
2023-11-22 08:31:10,684 - BERTopic - Clustered reduced embeddings
2023-11-22 08:32:32,628 - BERTopic - Reduced number of topics from 206 to 50


### 4) 버토픽 NPMI 최적화

min topic number가 NPMI기준 최적은 40개였으나, 토픽 distance map기준으로 봤을때 가장 고르게 된 것이 50개 였음으로 50으로 선정.

In [ ]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from sklearn.feature_extraction.text import CountVectorizer
from functools import partial
import numpy as np

# KeyBERT 및 MMR 모델 초기화
keybert_model = KeyBERTInspired()
mmr_model = MaximalMarginalRelevance(diversity=0.7)
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model
}

# 주어진 tokenizer 함수에 unique_values를 추가
tokenize_with_unique_values = partial(tokenize_jp_v2, unique_values=unique_values)

# 수정된 tokenizer를 CountVectorizer에 사용
vectorizer = CountVectorizer(tokenizer=tokenize_with_unique_values)
X = vectorizer.fit_transform(text)
word_freq = dict(zip(vectorizer.get_feature_names_out(), X.sum(axis=0).tolist()[0]))

# NPMI 최적화
best_topic_num = 0
best_npmi_score = -1

for num_topics in range(10, 101, 5):
    # BERTopic 모델을 초기화하고 토픽 수를 설정
    model = BERTopic(min_topic_size=40, nr_topics=num_topics, language="japanese", representation_model=representation_model, calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer)
    topics, _ = model.fit_transform(text)

    topic_npmi_scores = []
    unique_topics = list(set(topics))  # 중복 제거

    for unique_topic in unique_topics:
        if unique_topic == -1:  # Outlier는 무시
            continue

        topic_words = [word[0] for word in model.get_topic(unique_topic)]
        npmi_score = 0

        for word1 in topic_words:
            for word2 in topic_words:
                if word1 != word2 and word1 in word_freq and word2 in word_freq:
                    joint_prob = (X[:, vectorizer.vocabulary_[word1]].sum() * X[:, vectorizer.vocabulary_[word2]].sum()) / X.sum() ** 2
                    ind_prob = word_freq[word1] / X.sum() * word_freq[word2] / X.sum()
                    if joint_prob == 0 or ind_prob == 0:  # 로그 계산을 위한 예외 처리
                        continue
                    pmi = np.log(joint_prob / ind_prob)
                    n_pmi = pmi / -np.log(joint_prob)
                    npmi_score += n_pmi

        npmi_score /= len(topic_words) ** 2
        topic_npmi_scores.append(npmi_score)

    average_npmi = np.mean(topic_npmi_scores)

    if average_npmi > best_npmi_score:
        best_npmi_score = average_npmi
        best_topic_num = num_topics

print(f"Best Number of Topics: {best_topic_num} with NPMI = {best_npmi_score:.4f}")

Batches:   0%|          | 0/891 [00:00<?, ?it/s]

2023-10-26 05:33:16,011 - BERTopic - Transformed documents to Embeddings
2023-10-26 05:33:32,824 - BERTopic - Reduced dimensionality
2023-10-26 05:33:39,668 - BERTopic - Clustered reduced embeddings
2023-10-26 05:34:44,329 - BERTopic - Reduced number of topics from 54 to 10


Batches:   0%|          | 0/891 [00:00<?, ?it/s]

2023-10-26 05:35:30,179 - BERTopic - Transformed documents to Embeddings
2023-10-26 05:35:45,243 - BERTopic - Reduced dimensionality
2023-10-26 05:35:51,406 - BERTopic - Clustered reduced embeddings
2023-10-26 05:36:57,523 - BERTopic - Reduced number of topics from 54 to 15


Batches:   0%|          | 0/891 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/891 [00:00<?, ?it/s]

2023-10-26 06:15:56,492 - BERTopic - Transformed documents to Embeddings
2023-10-26 06:16:11,719 - BERTopic - Reduced dimensionality
2023-10-26 06:16:17,166 - BERTopic - Clustered reduced embeddings
2023-10-26 06:16:58,331 - BERTopic - Reduced number of topics from 51 to 51


Batches:   0%|          | 0/891 [00:00<?, ?it/s]

2023-10-26 06:18:19,000 - BERTopic - Transformed documents to Embeddings
2023-10-26 06:18:34,126 - BERTopic - Reduced dimensionality
2023-10-26 06:18:40,235 - BERTopic - Clustered reduced embeddings
2023-10-26 06:19:19,707 - BERTopic - Reduced number of topics from 57 to 57


Best Number of Topics: 40 with NPMI = -0.2844


### 5) 버토픽 최종 모델 로드 후 통합

In [ ]:
import pickle
from bertopic import BERTopic
import torch

my_model = BERTopic.load("/content/drive/MyDrive/BA_capstone/my_model")
new_topics, new_probs = my_model.transform(text)

In [ ]:
#덴드로그램 시각화
from scipy.cluster import hierarchy as sch
linkage_function = lambda x: sch.linkage(x, 'ward', optimal_ordering=True)
hierarchical_topics = topic_model.hierarchical_topics(text,linkage_function=linkage_function)

In [ ]:
my_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
#@title 토픽 통합

topics_to_merge = [[48,33,47,35,28,36,7,37,24],
                   [11,42,46,41,22,43],
                   [38,6,21,25,14,26,44],
                   [23,32],
                   [1,40,19],
                    [8,12,15,2,0,16,4,27],
                   [9,10,5,17],
                   [30,34,39,13,20],
                    [31,3,29,45,18]
                   ]
my_model.merge_topics(text, topics_to_merge)
new_topics, new_probs = my_model.transform(text)

### 6) 최종모델 시각화

In [ ]:
#@title Topic Count
display(my_model.get_topic_info().style.set_properties(**{'text-align': 'left'}))

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,14412,-1_ニキビ_化粧水_美容液_商品,"['ニキビ', '化粧水', '美容液', '商品', 'オススメ', '効果', '毛穴', '乳液', '皮膚', 'クリーム']","['導入美容液', '化粧水', '皮膚', '乾燥肌', '敏感肌', '美容液', '混合肌', '肌荒れ', 'クリーム', 'メイクアップ']","['ニキビ', '化粧水', '美容液', '商品', 'オススメ', '効果', '毛穴', '乳液', '皮膚', 'クリーム']","['いただいたので使いたいのですが、化粧水の前ですかあとですかニキビ跡にも効果的なのでしょうか', '最近ニキビができやすくなってきました。ニキビができた時、どのようなことをしていましたか。また、ニキビができた時にオススメな乳液や化粧水、洗顔などを教えてください', '美容液でも化粧水でもクリームでも、美白効果のあるいいスキンケア商品を教えてくださいシミやくすみが気になっています。']"
1,0,4599,0_化粧水_乳液_オススメ_美容液,"['化粧水', '乳液', 'オススメ', '美容液', 'ビタミン', '乾燥肌', '敏感肌', 'クリーム', '商品', 'アルビオン']","['化粧水', '導入美容液', '乳液', '乾燥肌', 'ミルク', 'クリーム', '美容液', '皮膚', '肌荒れ', '混合肌']","['化粧水', '乳液', 'オススメ', '美容液', 'ビタミン', '乾燥肌', '敏感肌', 'クリーム', '商品', 'アルビオン']","['化粧水、美容液のあとに乳液かクリームどちらかじゃだめなんでしょうか無知ですみません', '化粧水使わずに乳液使ってる方いますか', '化粧水と乳液で、配合されていて、良かったと思うものあれば教えてください。']"
2,1,1836,1_オイル_クレンジング_オイルクレンジング_日焼け止め,"['オイル', 'クレンジング', 'オイルクレンジング', '日焼け止め', '風呂', 'オススメ', 'メイクアップ', 'タイプ', '毛穴', 'ダブル洗顔']","['オイルクレンジング', '乾燥肌', 'クリーム', '化粧水', '肌荒れ', '日焼け止め', 'アルガンオイル', 'オイル', '洗顔フォーム', '水洗顔']","['オイル', 'クレンジング', 'オイルクレンジング', '日焼け止め', '風呂', 'オススメ', 'メイクアップ', 'タイプ', '毛穴', 'ダブル洗顔']","['メイクアップは日焼け止めのみ使用することが多いため、今まではメイクアップも落とせる洗顔料や、洗顔料のみで洗顔を済ませていました。しかし、これでは日焼け止めはきちんと落とせていないのでしょうか洗顔に加えてクレンジングも行うべきでしょうかまた、その場合、オイルやジェル、ミルクなどオススメのタイプのクレンジングがありましたら教えてください。', 'クレンジングをした後に肌がつっぱるかんじがするのですがこれは肌にあっていないということでしょうか…オイルタイプのものを使っています', 'よく肌を擦ってはいけないと聞くのですが、クレンジングは擦らないオイルがいいのか、それとも肌に負担をかけないと言われているクリームクレンジングがいいのか迷います。']"
3,2,1536,2_マスク_韓国_香り_オススメ,"['マスク', '韓国', '香り', 'オススメ', '花粉', '商品', '肌荒れ', 'リップ', 'ニキビ', '匂い']","['マスク荒れ', 'ピングマスク', 'マスク', '皮膚', '乾燥肌', '肌荒れ', 'メイクアップ', '敏感肌', 'パワーマスク', '化粧水']","['マスク', '韓国', '香り', 'オススメ', '花粉', '商品', '肌荒れ', 'リップ', 'ニキビ', '匂い']","['マスクでニキビができやすくなりました。オススメの化粧品教えてください', '仕事などで長時間つけていても肌が荒れにくいオススメの不織布マスクがあれば教えていただきたいです', 'マスク生活で頬と顎にニキビが大量に出来ました。オススメのスキンケアあれば教えて欲しいです']"
4,3,1448,3_毛穴_黒ずみ_酵素_小鼻,"['毛穴', '黒ずみ', '酵素', '小鼻', '効果', 'ケア', 'いちご', 'オススメ', '開き', '商品']","['酵素洗顔パウダー', '洗顔フォーム', '化粧水', '乾燥肌', '酵素', '乳液', '混合肌', '皮膚', 'ニキビ', '汚れ']","['毛穴', '黒ずみ', '酵素', '小鼻', '効果', 'ケア', 'いちご', 'オススメ', '開き', '商品']","['酵素洗顔以外で黒ずみ毛穴が綺麗になる商品ありますか', '最近鼻の黒ずみが気になって角栓を出してしまい余計に鼻の毛穴のへこみが目立つようになってしまったのですが、どうしたら毛穴を少しでも小さくすることが出来ますかまたオススメの洗顔や気をつけた方がいい事など、教えて頂けると嬉しいです。', '私は、ずっと毛穴の黒ずみに悩んでいての酵素洗顔を0ヶ月使用していましたが、効果を感じられませんでした…。黒ずみの毛穴に、悩んでいた方で今は、きれいになった方で効果を感じたと思う商品を教えて欲しいです']"
5,4,1408,4_クリーム_アイクリーム_オススメ_シワ,"['クリーム', 'アイクリーム', 'オススメ', 'シワ', '目元', 'フェイスクリーム', 'クマ', 'ハンド', '効果', '目の下']","['クリーム', 'リンクルクリーム', 'リップクリーム', 'フェイスクリーム', 'シカクリーム', '乾燥肌', 'アイクリーム', '香料', 'ジェルクリーム', '混合肌']","['クリーム', 'アイクリーム', 'オススメ', 'シワ', '目元', 'フェイスクリーム', 'クマ', 'ハンド', '効果', '目の下']","['肌がめちゃくちゃ乾燥で、オススメのクリームありましたら教えていただきたいです', '化粧品を愛する皆様こんにちは。皆さんが使ってて良いなと思う朝用クリーム教えて下さいクリーム使われていない方はオススメのスキンケア商品を教えて下さい。', '片方のみ三重になりました。オススメのクリームなど何か良いものがあれば教えてください。']"
6,5,1405,5_商品_オススメ_サンプル_お福分け,"['商品', 'オススメ', 'サンプル', 'お福分け', 'パック', '効果', 'サイト', 'ブランド', 'クーポン', 'くじ']","['メイクアップ', '商品', 'オススメ', '福袋', '美容', 'ベスト', 'パック', 'クーポン', 'ブランド', 'スチーマー']","['商品', 'オススメ', 'サンプル', 'お福分け', 'パック', '効果', 'サイト', 'ブランド', 'クーポン', 'くじ']","['オススメのスキンケア商品教えてください。', ' 0からのスキンケアでオススメの商品はありますか', 'オススメのスキンケア商品があれば教えていただきたいです']"
7,6,827,6_まつ毛_レーザー_産毛_オススメ,"['まつ毛', 'レーザー', '産毛', 'オススメ', 'シミ', '効果', '美容液', '医療', '眉毛', 'シェーバー']","['髪の毛', 'ヘア', 'ヘアケア', '産毛', 'まつげ美容液', '前髪', '美容液', 'まつ毛', '化粧水', '眉毛']","['まつ毛', 'レーザー', '産毛', 'オススメ', 'シミ', '効果', '美容液', '医療', '眉毛', 'シェーバー']","[' 00円以内で購入できるオススメのまつ毛美容液ありますか', 'オススメのまつ毛美容液教えてください。', '実際に使ってみて伸びた効果のある、オススメのまつ毛美容液を教えてください']"
8,7,755,7_ニキビ_皮膚_赤み_おでこ,"['ニキビ', '皮膚', '赤み', 'おでこ', '効果', '毛穴', 'ケア', '副作用', '商品', 'オススメ']","['塗り薬', '肌荒れ', '皮膚', 'ニキビ', '敏感肌', 'メイクアップ', '腫れ', '蕁麻疹', '美容液', '化粧水']","['ニキビ', '皮膚', '赤み', 'おでこ', '効果', '毛穴', 'ケア', '副作用', '商品', 'オススメ']","['ニキビ跡に効くもの教えてください', 'ニキビがなかなか無くなりません皮膚科に行ったのですが、赤ニキビがなかなか無くなりません。なにかいいスキンケアなどがあれば、教えていただきたいです', '赤みのあるニキビ跡に悩んでいます。赤ニキビにも悩んでいて皮膚科にも0年程通っています。ニキビに塗る薬は貰いましたがニキビ跡は治っていくのを待ってと言われました。ですが、少しでもニキビ跡の赤みを薄くしたいです。オススメの商品があれば教えてほしいです。']"
9,8,270,8_ビーエー_連休_選手_お

In [ ]:
#@title Topic Word Scores Visualize
#@markdown class-based TF-IDF를 이용하여, 각 토픽を利用し、各トピックの単語の重要度を可視化 (デフォルトは最大8トピックまで表示)。
my_model.visualize_barchart(top_n_topics=10,n_words=10)


In [ ]:
#@title Topic Word Scores
my_model.get_topics(10)


{'Main': {-1: [('ニキビ', 0.05231019963812174),
   ('化粧水', 0.048897440545990975),
   ('美容液', 0.042463660787396225),
   ('商品', 0.039805690633740023),
   ('オススメ', 0.03919109197195461),
   ('効果', 0.038194124005679786),
   ('毛穴', 0.03558475054244357),
   ('乳液', 0.03422705851035686),
   ('皮膚', 0.03125215941400434),
   ('クリーム', 0.03113085072280351)],
  0: [('化粧水', 0.11309429727227323),
   ('乳液', 0.09731652549566001),
   ('オススメ', 0.06247013282800695),
   ('美容液', 0.059967116963476036),
   ('ビタミン', 0.05873545834375879),
   ('乾燥肌', 0.0445427422357841),
   ('敏感肌', 0.03633325699119486),
   ('クリーム', 0.034430173054960926),
   ('商品', 0.03350628098401041),
   ('アルビオン', 0.03218682172494596)],
  1: [('オイル', 0.15299093104835532),
   ('クレンジング', 0.1100102916543534),
   ('オイルクレンジング', 0.08431755638471689),
   ('日焼け止め', 0.08134248487974965),
   ('風呂', 0.06286200220671054),
   ('オススメ', 0.054863329409735255),
   ('メイクアップ', 0.04725832513400522),
   ('タイプ', 0.04313817898842332),
   ('毛穴', 0.030300311493599904),
   (

In [ ]:
#@markdown トピックは、最も代表的な単語から始まるいくつかの単語で表現される。各単語はc-TF-IDFスコアで表現される。スコアが高いほど、トピックを代表する単語を意味する。トピックの単語はc-TF-IDFスコアでソートされているため、単語を追加するごとにスコアは徐々に低下する。ある時点で、トピックの表現に単語を追加しても、c-TF-IDFスコアの合計がわずかに増加するだけになる。この効果を視覚化するために、各トピックのc-TF-IDFスコアを各単語の順位でプロットすることができる。つまり、c-TF-IDFスコアが最も高い単語がランク1となる単語の位置（単語ランク）をx軸に、Y軸にc-TF-IDFスコアを配置すると、トピック表現に単語を追加したときにc-TF-IDFスコアが低下していることを視覚化できる。これにより、エルボー法を用いて、トピックに含まれる最適な単語数が選択できる。
my_model.visualize_term_rank()

In [ ]:
my_model.visualize_topics()

In [ ]:
#@title HeatMap 표시
my_model.visualize_heatmap()

In [ ]:
#@title 데이터프레임 표시
import numpy as np
# 각 문서의 최대 확률값을 계산합니다.
max_probs_list = np.max(new_probs, axis=1)  # probs가 2차원 배열인 경우, 각 행의 최대값을 찾습니다.
text_list = text.to_list()

# 데이터 검증: 모든 리스트의 길이가 동일한지 확인합니다.
if not (len(text_list) == len(new_topics) == len(max_probs_list)):
    raise ValueError("All input lists must have the same length.")

number_of_shown = 16 #@param {type:"slider", min:5, max:20, step:1}

# 데이터 프레임을 생성합니다.
result_df2 = pd.DataFrame(
    {
        "text": text_list,
        "topic_no": new_topics ,
        "proba": max_probs_list,
    }
)

# 데이터 프레임 출력 설정을 조정합니다.
# pd.set_option('display.max_colwidth', None)

# 결과를 출력합니다.
result_df2.head(number_of_shown)
result_df2.to_csv('./df2.csv',encoding='utf-8-sig',index=False)

,text,topic_no,proba
0,朝はビタミン美容液とセラミド配合の美容液夜はレチノール美容液とセラミド配合の美容液を使ってい...,0,0.097811
1,乾燥肌なのですが、クレンジングに迷っています。乾燥肌だからクレンジングクリームを使っていたの...,-1,0.096239
2,アビブの復活草クリームを買おうと思っているのですが、メイクアップ前でも使えますか,4,0.868095
3,手持ちの化粧水でシートパックをしたいので、ストレッチが効いて厚手のシートが欲しいのですが、お...,-1,0.157564
4,クレンジングについての回答ありがとうございました。カウブランドがいいと大半の方が答えて頂きま...,-1,0.727415
5,洗い上がりがつるつるになるオススメ洗顔があれば教えて下さい,1,1.000000
6,ザダーママスクビタミンとレチノール美容液を使用しているので、ビタミン無配合パックを探し...,0,0.614421
7,昨日から、朝ビタミン0夜レチノール0を使い始めました。朝顔を見てみると、ハリが出ていて上向き...,0,0.137296
8,純石鹸だけで洗顔して角栓は落ち着いたのでしょうか朝晩の純石鹸での洗顔でしょうか参考にさせてく...,-1,0.175069
9,オイル分の少ないニキビが悪化しづらい乳液を教えてほしいです,-1,0.083407


In [ ]:
#@title 시간에 따른 토픽 변화
topics_over_time = my_model.topics_over_time(text, dates_list, datetime_format="%Y-%m-%d", nr_bins=20)
my_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

20it [00:08,  2.36it/s]


In [ ]:
topic_distr, topic_token_distr = my_model.approximate_distribution(text,window=4)

In [ ]:
my_model.visualize_distribution(topic_distr[1000])